In [1]:
!pip install matplotlib torch scipy librosa cn2an opencc numpy h5py numba

DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
!cd ".\monolithic_align"
!python setup.py build_ext --inplace

The system cannot find the path specified.
python: can't open file 'D:\\vits-cantonese\\setup.py': [Errno 2] No such file or directory


In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

#hps = utils.get_hparams_from_file("./configs/cantonese_base.json")
hps = utils.get_hparams_from_file("./configs/hkilang.json")

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("./models/G.pth", net_g, None)

DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=1051)
           2	LOAD_FAST(arg=0, lineno=1054)
           4	LOAD_CONST(arg=1, lineno=1054)
           6	BINARY_SUBSCR(arg=None, lineno=1054)
           8	LOAD_FAST(arg=0, lineno=1054)
          10	LOAD_CONST(arg=2, lineno=1054)
          12	BINARY_SUBSCR(arg=None, lineno=1054)
          14	COMPARE_OP(arg=4, lineno=1054)
          16	LOAD_FAST(arg=0, lineno=1054)
          18	LOAD_CONST(arg=1, lineno=1054)
          20	BINARY_SUBSCR(arg=None, lineno=1054)
          22	LOAD_FAST(arg=0, lineno=1054)
          24	LOAD_CONST(arg=3, lineno=1054)
          26	BINARY_SUBSCR(arg=None, lineno=1054)
          28	COMPARE_OP(arg=5, lineno=1054)
          30	BINARY_AND(arg=None, lineno=1054)
          32	RETURN_VALUE(arg=None, lineno=1054)
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:state.pc_initial: State(pc_initial=0 nstack_

In [39]:
import opencc

# Original Setting with OpenCC
# t2s = opencc.OpenCC('t2s.json')
# input_text = t2s.convert("我而家搭緊車")
# stn_tst = get_text("[GD]" + input_text + "[GD]", hps)

# Cantonese
# stn_tst = get_text("nei˩˧ ʦou˨ mɛː˥ jɛː˩˧ hɐi˧˥ muːn˨˩ hɐu˧˥ tsɐu˧˥ lɐi˨˩ tsɐu˧˥ hɵy˧ aː˧", hps)
# Waitau
# stn_tst = get_text("niː˩˧ ʦuː˨ miː˥jɛː˩˧ ʦʰɵy˩˧ mʊŋ˨˩hɐu˥ ʦɐu˥ lɵy˨˩ ʦɐu˥ hyː˩˧ aː˨ .", hps)
# Hakka
stn_tst = get_text("ʦʰɔːŋ˨˩ ʦʰɛːn˨˩  miːn˨˩  ŋʲɛt̚˥˩ kɔːŋ˩˧ ...  ŋiː˨˩ siː˥˩ tʰiː˥˩ sɔːŋ˥˩ sɔːŋ˩˧", hps)


with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1.5)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

In [7]:
# save audio
import soundfile as sf
sf.write('audio.mp3', audio, 22050, format='mp3')